In [1]:
import os

# Set Java 17 for PySpark
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = '/opt/homebrew/opt/openjdk@17/bin:' + os.environ.get('PATH', '')

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/17 14:35:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = '/Users/zlatavorobeva/Documents/GitHub/SQL Int/SQL_Intermediate/Spark/Data'

In [3]:
json_df2_path = data_path + "/utilization.json"
df = spark.read.format("json").load(json_df2_path)

In [4]:
df.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [5]:
df.createOrReplaceTempView("utilization")

In [6]:
df_count = spark.sql("SELECT count(*) FROM utilization")
df_count.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [7]:
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [8]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      103|    8744|
|      104|    7366|
|      100|     391|
|      105|    1110|
|      101|    9808|
|      102|    8586|
|      112|    7425|
|      113|    9418|
|      110|    2826|
|      107|    5646|
|      111|    3093|
|      108|    8375|
|      109|    3129|
|      116|    1167|
|      114|    2128|
|      115|    5284|
|      120|    2733|
|      118|    7913|
|      117|    3605|
|      126|    6365|
+---------+--------+
only showing top 20 rows


In [9]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows


In [11]:
df_sql = spark.sql("SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      

In [ ]:
df.createOrReplaceTempView("utilization")

In [24]:
csv_df_path = data_path + "/server_name.csv"
df_server = spark.read.csv(csv_df_path, header=True)
df_server.show()

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
|      105| 105 Server|
|      106| 106 Server|
|      107| 107 Server|
|      108| 108 Server|
|      109| 109 Server|
|      110| 110 Server|
|      111| 111 Server|
|      112| 112 Server|
|      113| 113 Server|
|      114| 114 Server|
|      115| 115 Server|
|      116| 116 Server|
|      117| 117 Server|
|      118| 118 Server|
|      119| 119 Server|
+---------+-----------+
only showing top 20 rows


In [ ]:
df_server.createOrReplaceTempView("server_name")

In [15]:
df_count = spark.sql("SELECT DISTINCT server_id FROM utilization ORDER BY server_id")
df_count.show()

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
|      110|
|      111|
|      112|
|      113|
|      114|
|      115|
|      116|
|      117|
|      118|
|      119|
+---------+
only showing top 20 rows


In [17]:
spark.sql("SELECT max(server_id), min(server_id) FROM utilization").show()

+--------------+--------------+
|max(server_id)|min(server_id)|
+--------------+--------------+
|           149|           100|
+--------------+--------------+



In [25]:
spark.sql("SELECT * FROM server_name").show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [21]:
df_join = spark.sql("SELECT \
                    u.server_id, sn.server_name, u.session_count \
                    FROM utilization u \
                    INNER JOIN server_name sn \
                    ON u.server_id = sn.server_id")


df_join.show()                   


{"ts": "2025-12-17 14:44:27.605", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `sn`.`server_name` cannot be resolved. Did you mean one of the following? [`sn`.`server_id`, `u`.`server_id`, `sn`.`event_datetime`, `sn`.`free_memory`, `u`.`event_datetime`]. SQLSTATE: 42703", "context": {"errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o27.sql.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `sn`.`server_name` cannot be resolved. Did you mean one of the following? [`sn`.`server_id`, `u`.`server_id`, `sn`.`event_datetime`, `sn`.`free_memory`, `u`.`event_datetime`]. SQLSTATE: 42703; line 1 pos 40;\n'Project [server_id#9L, 'sn.server_name, session_count#10L]\n+- Join Inner, (server_id#9L = server_id#233L)\n   :- SubqueryAlias u

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `sn`.`server_name` cannot be resolved. Did you mean one of the following? [`sn`.`server_id`, `u`.`server_id`, `sn`.`event_datetime`, `sn`.`free_memory`, `u`.`event_datetime`]. SQLSTATE: 42703; line 1 pos 40;
'Project [server_id#9L, 'sn.server_name, session_count#10L]
+- Join Inner, (server_id#9L = server_id#233L)
   :- SubqueryAlias u
   :  +- SubqueryAlias utilization
   :     +- View (`utilization`, [cpu_utilization#6, event_datetime#7, free_memory#8, server_id#9L, session_count#10L])
   :        +- Relation [cpu_utilization#6,event_datetime#7,free_memory#8,server_id#9L,session_count#10L] json
   +- SubqueryAlias sn
      +- SubqueryAlias server_name
         +- View (`server_name`, [cpu_utilization#230, event_datetime#231, free_memory#232, server_id#233L, session_count#234L])
            +- Relation [cpu_utilization#230,event_datetime#231,free_memory#232,server_id#233L,session_